# Protein Sequence Extraction from Genomic DNA

**Goal:** Extract protein sequences for all 4 psilocybin BGC genes (PsiD, PsiK, PsiM, PsiH) from genomic DNA scaffolds

**Workflow:** Replicate the Colab workflow from `01_Colab_codes_InputFiles/Codes/`

1. ✅ Install tools (already done - conda environment `AncSeq`)
2. Check input files
3. Run exonerate (protein2genome) to extract CDS
4. Translate CDS to protein
5. Quality control
6. Save results

---

## Setup

In [1]:
import os
import subprocess
from pathlib import Path
from Bio import SeqIO
from tqdm.notebook import tqdm
import time
import multiprocessing as mp
from functools import partial
import pandas as pd

# Set paths
PROJECT_DIR = Path.cwd().parent
INPUT_DIR = PROJECT_DIR / "01_Colab_codes_InputFiles" / "Files_paper_P_baeocystis"
EXAMPLE_OUTPUT = PROJECT_DIR / "02_ColabResults_Example"
RESULTS_DIR = Path.cwd() / "results"
RESULTS_DIR.mkdir(exist_ok=True)

print(f"Project directory: {PROJECT_DIR}")
print(f"Input directory: {INPUT_DIR}")
print(f"Results directory: {RESULTS_DIR}")
print(f"\nExample output directory: {EXAMPLE_OUTPUT}")
print(f"CPU cores available: {mp.cpu_count()}")

Project directory: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix
Input directory: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/01_Colab_codes_InputFiles/Files_paper_P_baeocystis
Results directory: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/results

Example output directory: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/02_ColabResults_Example
CPU cores available: 12


## Step 1: Check Input Files

Based on `02_check_files.txt`

In [2]:
# List files in input directory
!ls -lh "$INPUT_DIR"

# Expected files:
# 1. Reference protein: PsiD_Psilocybe_cubensis_reference.faa
# 2. Genome scaffolds: Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta
# 3. (Optional) GFF from paper: Psilocybe_baeocystis_WTU-F-011245-redundands.gff

total 282760
-rw-rw-rw-  1 felix  staff   505B Sep 19 00:21 PsiD_Psilocybe_cubensis_reference.faa
-rw-rw-rw-  1 felix  staff    32M Sep 24 16:07 Psilocybe_baeocystis_WTU-F-011245-redundands.gff
-rw-rw-rw-  1 felix  staff   106M Sep 24 16:16 Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta


In [3]:
# Check reference protein file
reference_protein = INPUT_DIR / "PsiD_Psilocybe_cubensis_reference.faa"
print(f"Reference protein exists: {reference_protein.exists()}")

if reference_protein.exists():
    for record in SeqIO.parse(reference_protein, "fasta"):
        print(f"\nID: {record.id}")
        print(f"Description: {record.description}")
        print(f"Length: {len(record.seq)} aa")
        print(f"Sequence preview: {str(record.seq)[:70]}...")

Reference protein exists: True

ID: ASU62239.1
Description: ASU62239.1 tryptophan decarboxylase [Psilocybe cubensis]
Length: 439 aa
Sequence preview: MQVIPACNSAAIRSLCPTPESFRNMGWLSVSDAVYSEFIGELATRASNRNYSNEFGLMQPIQEFKAFIES...


In [4]:
# Check genome scaffold file
genome_scaffolds = INPUT_DIR / "Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta"
print(f"Genome scaffolds exist: {genome_scaffolds.exists()}")

if genome_scaffolds.exists():
    # Count scaffolds
    scaffold_count = 0
    total_length = 0
    for record in SeqIO.parse(genome_scaffolds, "fasta"):
        scaffold_count += 1
        total_length += len(record.seq)
    
    print(f"Number of scaffolds: {scaffold_count}")
    print(f"Total genome length: {total_length:,} bp")
    print(f"Average scaffold size: {total_length // scaffold_count:,} bp")

Genome scaffolds exist: True
Number of scaffolds: 111256
Total genome length: 105,544,072 bp
Average scaffold size: 948 bp


## Step 2: Run Exonerate (Extract CDS)

Based on `03_exonerate_best_hit.txt`

This extracts the coding sequence (CDS) by aligning the reference protein to the genome scaffolds.

In [5]:
# Define output files
output_cds = RESULTS_DIR / "P_baeocystis_PsiD.cds.fa"
output_gff = RESULTS_DIR / "P_baeocystis_PsiD.gff"

# Command 1: Extract CDS
cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">PsiD|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''

print("Running exonerate to extract CDS...")
print(f"Command: {cmd_cds}\n")

# Run command
result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ CDS extraction complete!")
    print(f"  Output: {output_cds}")
else:
    print(f"✗ Error: {result.stderr}")

Running exonerate to extract CDS...
Command: exonerate --model protein2genome   "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/01_Colab_codes_InputFiles/Files_paper_P_baeocystis/PsiD_Psilocybe_cubensis_reference.faa"   "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/01_Colab_codes_InputFiles/Files_paper_P_baeocystis/Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta"   --bestn 1   --showalignment no --showvulgar no --verbose 0   --ryo ">PsiD|%ti:%tab-%tae(%tS)\n%tcs\n" > "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/results/P_baeocystis_PsiD.c

In [6]:
# Command 2: Extract GFF annotation
cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''

print("Running exonerate to extract GFF...")
print(f"Command: {cmd_gff}\n")

# Run command
result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ GFF extraction complete!")
    print(f"  Output: {output_gff}")
else:
    print(f"✗ Error: {result.stderr}")

Running exonerate to extract GFF...
Command: exonerate --model protein2genome   "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/01_Colab_codes_InputFiles/Files_paper_P_baeocystis/PsiD_Psilocybe_cubensis_reference.faa"   "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/01_Colab_codes_InputFiles/Files_paper_P_baeocystis/Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta"   --bestn 1   --showalignment no --showvulgar no --verbose 0   --showtargetgff yes > "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/results/P_baeocystis_PsiD.gff"

✓ GFF extraction

In [7]:
# Check the extracted CDS
print("Extracted CDS:")
if output_cds.exists():
    for record in SeqIO.parse(output_cds, "fasta"):
        print(f"  ID: {record.id}")
        print(f"  Length: {len(record.seq)} bp")
        print(f"  Sequence preview: {str(record.seq)[:70]}...")
else:
    print("  CDS file not found!")

Extracted CDS:
  ID: PsiD|NODE_6810_length_3081_cov_44.967840:3079-2461(-)
  Length: 618 bp
  Sequence preview: GACACCACTCTCATTTCGGCTGCCTGTGAATCACTTTCGTATAACGTCTCTCACAATGTCCAATCTCTCG...


## Step 3: Translate CDS to Protein

Based on `04_translate.txt`

Use EMBOSS transeq to translate the CDS to protein sequence.

In [8]:
# Define output protein file
output_protein = RESULTS_DIR / "P_baeocystis_PsiD.prot.fa"

# Command: Translate CDS to protein (frame 1)
cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'

print("Translating CDS to protein...")
print(f"Command: {cmd_translate}\n")

# Run command
result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ Translation complete!")
    print(f"  Output: {output_protein}")
else:
    print(f"✗ Error: {result.stderr}")

Translating CDS to protein...
Command: transeq -sequence "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/results/P_baeocystis_PsiD.cds.fa" -outseq "/Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/results/P_baeocystis_PsiD.prot.fa" -frame 1

✓ Translation complete!
  Output: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/results/P_baeocystis_PsiD.prot.fa


In [9]:
# Check the translated protein
print("Translated Protein:")
if output_protein.exists():
    for record in SeqIO.parse(output_protein, "fasta"):
        print(f"  ID: {record.id}")
        print(f"  Length: {len(record.seq)} aa")
        print(f"  Sequence: {str(record.seq)}")
else:
    print("  Protein file not found!")

Translated Protein:
  ID: EMBOSS_001_1
  Length: 206 aa
  Sequence: DTTLISAACESLSYNVSHNVQSLDTLFIKGEAYSLKHLLHNDPFTPQFEHGSIIQGFLNVTAYHRWHAPVNGTILKIVNVPGTYFAQAPSTIGEPIPDNDFDPPPYLKSLIYFSNIAARQIMFIEADNKDIGLIFLVFIGMTEISTCEATVSEGQHVNRGDDLGMFHFGGSSFALGLRKDCTAEIFETFKESGTVIKINDLVAAVK


## Step 4: Quality Control

Based on `05_quality_control.txt`

Check for:
1. Internal stop codons (should be none)
2. Protein length

In [10]:
# Check for internal stop codons
print("=" * 60)
print("QUALITY CONTROL")
print("=" * 60)

if output_protein.exists():
    for record in SeqIO.parse(output_protein, "fasta"):
        sequence = str(record.seq)
        
        # Check for internal stops (excluding terminal *)
        internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
        stop_count = internal_seq.count('*')
        
        if stop_count == 0:
            print("✅ No internal stop codons found")
        else:
            print(f"⚠️  WARNING: {stop_count} internal stop codon(s) found!")
            # Show positions
            for i, aa in enumerate(internal_seq):
                if aa == '*':
                    print(f"    Position {i+1}: STOP")
        
        # Check protein length
        protein_length = len(sequence)
        print(f"\nProtein length: {protein_length} aa")
        
        # Compare to reference
        for ref_record in SeqIO.parse(reference_protein, "fasta"):
            ref_length = len(ref_record.seq)
            print(f"Reference length: {ref_length} aa")
            
            diff = protein_length - ref_length
            if diff == 0:
                print("✅ Length matches reference exactly")
            else:
                print(f"⚠️  Length difference: {diff:+d} aa ({abs(diff/ref_length*100):.1f}%)")

QUALITY CONTROL
✅ No internal stop codons found

Protein length: 206 aa
Reference length: 439 aa
⚠️  Length difference: -233 aa (53.1%)


In [11]:
# Preview the first few lines of the protein file
print("\nProtein FASTA file preview:")
!head -n 5 "{output_protein}"


Protein FASTA file preview:
>EMBOSS_001_1
DTTLISAACESLSYNVSHNVQSLDTLFIKGEAYSLKHLLHNDPFTPQFEHGSIIQGFLNV
TAYHRWHAPVNGTILKIVNVPGTYFAQAPSTIGEPIPDNDFDPPPYLKSLIYFSNIAARQ
IMFIEADNKDIGLIFLVFIGMTEISTCEATVSEGQHVNRGDDLGMFHFGGSSFALGLRKD
CTAEIFETFKESGTVIKINDLVAAVK


## Step 5: Compare with Example Output

Verify our results match the example from `02_ColabResults_Example/`

In [12]:
# Compare ALL output files with examples
print("Comparing with example output:")
print("=" * 60)

# 1. Compare PROTEIN sequences
example_protein = EXAMPLE_OUTPUT / "P_baeocystis_PsiD.prot.fa"
print("\n1. PROTEIN COMPARISON:")
if example_protein.exists() and output_protein.exists():
    our_seq = str(list(SeqIO.parse(output_protein, "fasta"))[0].seq)
    example_seq = str(list(SeqIO.parse(example_protein, "fasta"))[0].seq)
    
    print(f"   Our length: {len(our_seq)} aa")
    print(f"   Example length: {len(example_seq)} aa")
    
    if our_seq == example_seq:
        print("   ✅ PROTEIN: Perfect match!")
    else:
        print("   ⚠️  PROTEIN: Sequences differ")
        for i, (aa1, aa2) in enumerate(zip(our_seq, example_seq)):
            if aa1 != aa2:
                print(f"      Position {i+1}: {aa1} vs {aa2}")

# 2. Compare CDS sequences
example_cds = EXAMPLE_OUTPUT / "P_baeocystis_PsiD.cds.fa"
print("\n2. CDS COMPARISON:")
if example_cds.exists() and output_cds.exists():
    our_cds = str(list(SeqIO.parse(output_cds, "fasta"))[0].seq)
    example_cds_seq = str(list(SeqIO.parse(example_cds, "fasta"))[0].seq)
    
    print(f"   Our length: {len(our_cds)} bp")
    print(f"   Example length: {len(example_cds_seq)} bp")
    
    if our_cds == example_cds_seq:
        print("   ✅ CDS: Perfect match!")
    else:
        print(f"   ⚠️  CDS: Sequences differ ({sum(1 for a,b in zip(our_cds, example_cds_seq) if a!=b)} differences)")

# 3. Compare GFF files (just check if they exist and basic structure)
example_gff = EXAMPLE_OUTPUT / "P_baeocystis_PsiD.gff"
print("\n3. GFF COMPARISON:")
if example_gff.exists() and output_gff.exists():
    our_gff_lines = open(output_gff).readlines()
    example_gff_lines = open(example_gff).readlines()
    
    print(f"   Our GFF lines: {len(our_gff_lines)}")
    print(f"   Example GFF lines: {len(example_gff_lines)}")
    
    if our_gff_lines == example_gff_lines:
        print("   ✅ GFF: Perfect match!")
    else:
        print(f"   ⚠️  GFF: Files differ")

print("\n" + "=" * 60)
print("VERIFICATION COMPLETE")

Comparing with example output:

1. PROTEIN COMPARISON:
   Our length: 206 aa
   Example length: 206 aa
   ✅ PROTEIN: Perfect match!

2. CDS COMPARISON:
   Our length: 618 bp
   Example length: 618 bp
   ✅ CDS: Perfect match!

3. GFF COMPARISON:
   Our GFF lines: 17
   Example GFF lines: 17
   ⚠️  GFF: Files differ

VERIFICATION COMPLETE


## Summary

**Output files created:**
- CDS (DNA): `results/P_baeocystis_PsiD.cds.fa`
- Protein: `results/P_baeocystis_PsiD.prot.fa`
- GFF annotation: `results/P_baeocystis_PsiD.gff`

**Next steps:**
1. ✅ Verify this works for PsiD
2. Replicate for other genes (PsiK, PsiM, PsiH)
3. Batch process all species from `03_Sequences_Paper/`
4. Combine all extracted proteins for phylogenetic analysis

In [13]:
# If GFF differs, show the differences
if example_gff.exists() and output_gff.exists():
    our_gff_lines = open(output_gff).readlines()
    example_gff_lines = open(example_gff).readlines()
    
    if our_gff_lines != example_gff_lines:
        print("\nGFF Differences (first 5 differences shown):")
        print("-" * 60)
        diff_count = 0
        for i, (our_line, ex_line) in enumerate(zip(our_gff_lines, example_gff_lines)):
            if our_line != ex_line and diff_count < 5:
                print(f"\nLine {i+1}:")
                print(f"  Ours:    {our_line.strip()}")
                print(f"  Example: {ex_line.strip()}")
                diff_count += 1


GFF Differences (first 5 differences shown):
------------------------------------------------------------

Line 6:
  Ours:    ##date 2025-10-20
  Example: ##date 2025-10-01


---

# Extract All 4 BGC Enzymes

Now that we've validated the workflow with PsiD, let's extract all 4 enzymes from P. baeocystis:
1. **PsiD** – Tryptophan decarboxylase
2. **PsiK** – 4-Hydroxytryptamine kinase
3. **PsiM** – Psilocybin methyltransferase
4. **PsiH** – Tryptamine 4-monooxygenase

---

In [14]:
# Define all 4 BGC genes with their reference proteins
REF_PROTEIN_DIR = Path.cwd() / "reference_proteins"

BGC_GENES = {
    'PsiD': {
        'name': 'Tryptophan decarboxylase',
        'reference': INPUT_DIR / "PsiD_Psilocybe_cubensis_reference.faa",
        'function': 'Converts tryptophan to tryptamine'
    },
    'PsiK': {
        'name': '4-Hydroxytryptamine kinase',
        'reference': REF_PROTEIN_DIR / "PsiK_Psilocybe_cubensis_reference.faa",
        'function': 'Phosphorylates 4-hydroxytryptamine'
    },
    'PsiM': {
        'name': 'Psilocybin methyltransferase',
        'reference': REF_PROTEIN_DIR / "PsiM_Psilocybe_cubensis_reference.faa",
        'function': 'N-methylates norbaeocystin to baeocystin and psilocybin'
    },
    'PsiH': {
        'name': 'Tryptamine 4-monooxygenase',
        'reference': REF_PROTEIN_DIR / "PsiH_Psilocybe_cubensis_reference.faa",
        'function': 'Hydroxylates tryptamine at C-4 position'
    }
}

# Verify all reference files exist
print("Checking reference protein files:")
for gene, info in BGC_GENES.items():
    exists = info['reference'].exists()
    symbol = "✅" if exists else "❌"
    print(f"  {symbol} {gene}: {info['reference'].name}")

Checking reference protein files:
  ✅ PsiD: PsiD_Psilocybe_cubensis_reference.faa
  ✅ PsiK: PsiK_Psilocybe_cubensis_reference.faa
  ✅ PsiM: PsiM_Psilocybe_cubensis_reference.faa
  ✅ PsiH: PsiH_Psilocybe_cubensis_reference.faa


## PsiD - Tryptophan Decarboxylase

Extract PsiD protein from P. baeocystis genome

In [15]:
gene = 'PsiD'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

Processing PsiD - Tryptophan decarboxylase



PsiD:   0%|          | 0/4 [00:00]

   ✓ CDS extracted: P_baeocystis_PsiD.cds.fa
   ✓ GFF extracted: P_baeocystis_PsiD.gff
   ✓ Protein translated: P_baeocystis_PsiD.prot.fa

   Quality Control:
   ✅ No internal stop codons
   ⚠️  Length: 206 aa (ref: 439 aa, diff: -233)




## PsiK – 4-Hydroxytryptamine Kinase

Extract PsiK protein from P. baeocystis genome

In [16]:
gene = 'PsiK'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

Processing PsiK - 4-Hydroxytryptamine kinase



PsiK:   0%|          | 0/4 [00:00]

   ✓ CDS extracted: P_baeocystis_PsiK.cds.fa
   ✓ GFF extracted: P_baeocystis_PsiK.gff




## PsiM – Psilocybin Methyltransferase

Extract PsiM protein from P. baeocystis genome

In [17]:
gene = 'PsiM'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

Processing PsiM - Psilocybin methyltransferase



PsiM:   0%|          | 0/4 [00:00]

   ✓ CDS extracted: P_baeocystis_PsiM.cds.fa
   ✓ GFF extracted: P_baeocystis_PsiM.gff




## PsiH – Tryptamine 4-Monooxygenase

Extract PsiH protein from P. baeocystis genome

In [18]:
gene = 'PsiH'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

Processing PsiH - Tryptamine 4-monooxygenase



PsiH:   0%|          | 0/4 [00:00]

   ✓ CDS extracted: P_baeocystis_PsiH.cds.fa
   ✓ GFF extracted: P_baeocystis_PsiH.gff
   ✓ Protein translated: P_baeocystis_PsiH.prot.fa

   Quality Control:
   ✅ No internal stop codons
   ⚠️  Length: 61 aa (ref: 69 aa, diff: -8)




In [19]:
# DEBUG CELL - Add this after cell 32 in your notebook
# This will help us understand why PsiK and PsiM are not being found

import subprocess
from Bio import SeqIO

print("="*80)
print("DEBUGGING PsiK and PsiM EXTRACTION FOR P. baeocystis")
print("="*80)

# Test each gene with verbose exonerate output
for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    ref_file = BGC_GENES[gene]['reference']

    print(f"\n{'='*80}")
    print(f"Testing {gene} - {BGC_GENES[gene]['name']}")
    print(f"{'='*80}")

    # Read reference info
    ref_record = list(SeqIO.parse(ref_file, "fasta"))[0]
    print(f"\n📋 Reference protein:")
    print(f"   ID: {ref_record.id}")
    print(f"   Length: {len(ref_record.seq)} aa")
    print(f"   First 50 aa: {str(ref_record.seq)[:50]}...")

    # Run exonerate with alignment output to see what's happening
    output_file = RESULTS_DIR / f"debug_{gene}_verbose.txt"

    print(f"\n🔍 Running exonerate (this may take 10-20 seconds)...")
    cmd = f'''exonerate --model protein2genome \
      "{ref_file}" \
      "{genome_scaffolds}" \
      --bestn 1 \
      --showalignment yes \
      --showvulgar no \
      --verbose 0 \
      > "{output_file}"'''

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=60)

    # Analyze output
    if output_file.exists():
        with open(output_file, 'r') as f:
            output = f.read()

        # Check for matches
        if "C4 Alignment:" in output:
            print(f"   ✅ MATCH FOUND!")

            # Extract score and identity
            for line in output.split('\n'):
                if 'score' in line.lower():
                    print(f"   {line.strip()}")

            # Show a bit of the alignment
            lines = output.split('\n')
            for i, line in enumerate(lines):
                if "C4 Alignment:" in line:
                    print(f"\n   Alignment preview (first 10 lines):")
                    for j in range(i, min(i+10, len(lines))):
                        print(f"   {lines[j]}")
                    break
        else:
            print(f"   ❌ NO MATCH FOUND")
            print(f"   Output file size: {output_file.stat().st_size} bytes")
            if output_file.stat().st_size > 0:
                print(f"   First 500 chars of output:")
                print(f"   {output[:500]}")
    else:
        print(f"   ❌ Output file not created")

# Now try with more relaxed parameters for the failed genes
print(f"\n\n{'='*80}")
print("TRYING MORE RELAXED PARAMETERS FOR PsiK and PsiM")
print(f"{'='*80}")

for gene in ['PsiK', 'PsiM']:
    ref_file = BGC_GENES[gene]['reference']

    print(f"\n{gene}: Trying with --percent 30 (allow 30% identity)")

    output_file = RESULTS_DIR / f"debug_{gene}_relaxed.txt"
    cmd = f'''exonerate --model protein2genome \
      "{ref_file}" \
      "{genome_scaffolds}" \
      --bestn 3 \
      --percent 30 \
      --showalignment yes \
      --verbose 0 \
      > "{output_file}"'''

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=60)

    if output_file.exists():
        with open(output_file, 'r') as f:
            output = f.read()

        if "C4 Alignment:" in output:
            print(f"   ✅ MATCH FOUND with relaxed parameters!")
            # Count how many alignments
            alignment_count = output.count("C4 Alignment:")
            print(f"   Found {alignment_count} alignment(s)")

            for line in output.split('\n')[:50]:  # Show first 50 lines
                if 'score' in line.lower() or 'Query:' in line or 'Target:' in line:
                    print(f"   {line.strip()}")
        else:
            print(f"   ❌ Still no match even with relaxed parameters")
            print(f"   This suggests the gene may be genuinely absent or too divergent")

print(f"\n{'='*80}")
print("CONCLUSION")
print(f"{'='*80}")
print("\nIf PsiK and PsiM show 'NO MATCH FOUND' even with relaxed parameters,")
print("this confirms these genes are either:")
print("  1. Genuinely absent in P. baeocystis")
print("  2. Too divergent from P. cubensis reference (>70% divergence)")
print("  3. Split across multiple contigs in this assembly")


DEBUGGING PsiK and PsiM EXTRACTION FOR P. baeocystis

Testing PsiD - Tryptophan decarboxylase

📋 Reference protein:
   ID: ASU62239.1
   Length: 439 aa
   First 50 aa: MQVIPACNSAAIRSLCPTPESFRNMGWLSVSDAVYSEFIGELATRASNRN...

🔍 Running exonerate (this may take 10-20 seconds)...
   ✅ MATCH FOUND!
   Raw score: 981

   Alignment preview (first 10 lines):
   C4 Alignment:
   ------------
            Query: ASU62239.1 tryptophan decarboxylase [Psilocybe cubensis]
           Target: NODE_6810_length_3081_cov_44.967840 [revcomp]
            Model: protein2genome:local
        Raw score: 981
      Query range: 232 -> 438
     Target range: 3079 -> 2461
   
     233 : AsnThrThrLeuIleSerAlaAlaCysGluSerLeuSerTyrAsnValSerTyrAspValGl :  253

Testing PsiK - 4-Hydroxytryptamine kinase

📋 Reference protein:
   ID: ASU62240.1
   Length: 496 aa
   First 50 aa: MSPNDFRFFTKHLIDHSSHGAPLVPDQIHVDDKRFFRQLLANGVSAFECV...

🔍 Running exonerate (this may take 10-20 seconds)...
   ❌ NO MATCH FOUND
   Output file size

In [20]:
# Compare P. cubensis vs P. baeocystis references for PsiK and PsiM

import subprocess
from pathlib import Path

print("=" * 80)
print("COMPARING P. CUBENSIS vs P. BAEOCYSTIS REFERENCES")
print("=" * 80)

# Define both sets of references
test_configs = {
    'PsiK': {
        'cubensis': REF_PROTEIN_DIR / "PsiK_Psilocybe_cubensis_reference.faa",
        'baeocystis': REF_PROTEIN_DIR / "PsiK_Psilocybe_baeocystis_reference.faa"
    },
    'PsiM': {
        'cubensis': REF_PROTEIN_DIR / "PsiM_Psilocybe_cubensis_reference.faa",
        'baeocystis': REF_PROTEIN_DIR / "PsiM_Psilocybe_baeocystis_reference.faa"
    }
}

for gene, refs in test_configs.items():
    print(f"\n{'=' * 80}")
    print(f"TESTING {gene}")
    print(f"{'=' * 80}")

    for ref_type, ref_file in refs.items():
        print(f"\n--- Using {ref_type.upper()} reference ---")

        if not ref_file.exists():
            print(f"⚠️  Reference file not found: {ref_file}")
            print(f"   (Run previous cell first if testing baeocystis)")
            continue

        # Show reference info
        ref_record = list(SeqIO.parse(ref_file, "fasta"))[0]
        print(f"   Reference: {ref_record.id}")
        print(f"   Length: {len(ref_record.seq)} aa")

        # Run exonerate
        output_file = RESULTS_DIR / f"debug_{gene}_{ref_type}.cds.fa"

        cmd = f'''exonerate --model protein2genome \
          "{ref_file}" \
          "{genome_scaffolds}" \
          --bestn 1 \
          --showalignment no \
          --showvulgar no \
          --verbose 0 \
          --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_file}"'''

        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=60)

        # Check results
        if output_file.exists() and output_file.stat().st_size > 0:
            # Read CDS and translate
            cds_record = list(SeqIO.parse(output_file, "fasta"))[0]
            cds_seq = cds_record.seq
            protein_seq = cds_seq.translate()

            print(f"   ✅ MATCH FOUND with {ref_type} reference!")
            print(f"      Scaffold: {cds_record.id}")
            print(f"      CDS length: {len(cds_seq)} bp")
            print(f"      Protein length: {len(protein_seq)} aa")
            print(f"      First 50 aa: {protein_seq[:50]}")
        else:
            print(f"   ❌ NO MATCH with {ref_type} reference")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)
print("\n📊 Interpretation:")
print("   ✅ If P. baeocystis references work → Use them for batch processing")
print("   ❌ If neither works → Need to investigate paper's methodology further")

COMPARING P. CUBENSIS vs P. BAEOCYSTIS REFERENCES

TESTING PsiK

--- Using CUBENSIS reference ---
   Reference: ASU62240.1
   Length: 496 aa
   ❌ NO MATCH with cubensis reference

--- Using BAEOCYSTIS reference ---
⚠️  Reference file not found: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/reference_proteins/PsiK_Psilocybe_baeocystis_reference.faa
   (Run previous cell first if testing baeocystis)

TESTING PsiM

--- Using CUBENSIS reference ---
   Reference: ASU62241.1
   Length: 601 aa
   ❌ NO MATCH with cubensis reference

--- Using BAEOCYSTIS reference ---
⚠️  Reference file not found: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/reference_proteins/PsiM_Psilocybe_baeo

## Compare P. cubensis vs P. baeocystis References

Test whether P. baeocystis-derived references work better than P. cubensis GenBank references

In [21]:
## Extract P. baeocystis PsiK and PsiM from Paper's Data

# The paper successfully extracted these genes - let's use their sequences as references
from Bio import SeqIO

# Read PsiK and PsiM DNA sequences from paper's data
psik_file = PROJECT_DIR / "03_Sequences_Paper/DNA_All/PsiK/PsiK_exonerate_full_info.fasta"
psim_file = PROJECT_DIR / "03_Sequences_Paper/DNA_All/PsiM/PsiM_exonerate_full_info.fasta"

# Create reference directory
reference_dir = Path.cwd() / "reference_proteins"
reference_dir.mkdir(exist_ok=True)

print("Extracting P. baeocystis reference sequences from paper's data...\n")
print("="*80)

# Extract and translate PsiK
print("\n🧬 PsiK - 4-Hydroxytryptamine kinase")
print("-"*80)
for record in SeqIO.parse(psik_file, "fasta"):
    if "Psilocybe_baeocystis" in record.id:
        print(f"✓ Found in paper's data: {record.id}")
        print(f"  DNA length: {len(record.seq)} bp")
        
        # Translate to protein
        protein = record.seq.translate()
        print(f"  Protein length: {len(protein)} aa")
        print(f"  First 50 aa: {protein[:50]}")
        
        # Save to reference file
        output_file = reference_dir / "PsiK_Psilocybe_baeocystis_reference.faa"
        with open(output_file, "w") as f:
            f.write(f">PsiK_Psilocybe_baeocystis_from_paper\n{protein}\n")
        print(f"  ✅ Saved to: {output_file.name}\n")
        break

# Extract and translate PsiM
print("🧬 PsiM - Psilocybin methyltransferase")
print("-"*80)
for record in SeqIO.parse(psim_file, "fasta"):
    if "Psilocybe_baeocystis" in record.id:
        print(f"✓ Found in paper's data: {record.id}")
        print(f"  DNA length: {len(record.seq)} bp")
        
        # Translate to protein
        protein = record.seq.translate()
        print(f"  Protein length: {len(protein)} aa")
        print(f"  First 50 aa: {protein[:50]}")
        
        # Save to reference file
        output_file = reference_dir / "PsiM_Psilocybe_baeocystis_reference.faa"
        with open(output_file, "w") as f:
            f.write(f">PsiM_Psilocybe_baeocystis_from_paper\n{protein}\n")
        print(f"  ✅ Saved to: {output_file.name}\n")
        break

print("="*80)
print("✓ P. baeocystis reference files created successfully!")
print("\nNext: Test these references with exonerate to see if they work better.")

Extracting P. baeocystis reference sequences from paper's data...


🧬 PsiK - 4-Hydroxytryptamine kinase
--------------------------------------------------------------------------------
✓ Found in paper's data: Psilocybe_baeocystis_WTU-F-011245-redundands|Psi_K|NODE_8934_length_2116_cov_57.656611_(1540-387)_strand(+)
  DNA length: 1153 bp
  Protein length: 384 aa
  First 50 aa: MAFDLKTPEGLLLYLTRHLSLDVDPSGVKRLSGGFVNVTWRIRLNAPYQG
  ✅ Saved to: PsiK_Psilocybe_baeocystis_reference.faa

🧬 PsiM - Psilocybin methyltransferase
--------------------------------------------------------------------------------
✓ Found in paper's data: Psilocybe_baeocystis_WTU-F-011245-redundands|Psi_M|NODE_6690_length_3081_cov_44.967840_(979-0)_strand(+)
  DNA length: 979 bp
  Protein length: 326 aa
  First 50 aa: MHIRNPYRSPIDYQALVEAFPPLRP*CALILFSLPFIQI*STIV*QSIKT
  ✅ Saved to: PsiM_Psilocybe_baeocystis_reference.faa

✓ P. baeocystis reference files created successfully!

Next: Test these references with exonerate 

/opt/miniconda3/envs/AncSeq/lib/python3.9/site-packages/Bio/Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


---

# Fix PsiK and PsiM Extraction

The paper successfully extracted PsiK and PsiM from P. baeocystis, but our P. cubensis references aren't finding them. Let's extract P. baeocystis sequences from the paper's data and use those as references instead.

---

## Final Summary

Review all extracted proteins

In [22]:
import pandas as pd

# Collect results for all 4 genes
results = []

for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    protein_file = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"
    
    if protein_file.exists():
        record = list(SeqIO.parse(protein_file, "fasta"))[0]
        sequence = str(record.seq)
        internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
        
        # Get reference length
        ref_file = BGC_GENES[gene]['reference']
        ref_record = list(SeqIO.parse(ref_file, "fasta"))[0]
        
        results.append({
            'Gene': gene,
            'Enzyme': BGC_GENES[gene]['name'],
            'Length (aa)': len(sequence),
            'Reference (aa)': len(ref_record.seq),
            'Difference': len(sequence) - len(ref_record.seq),
            'Stop codons': internal_seq.count('*'),
            'Status': '✅' if internal_seq.count('*') == 0 else '⚠️'
        })

# Display as table
df = pd.DataFrame(results)
print("\n" + "="*80)
print("PROTEIN EXTRACTION SUMMARY - Psilocybe baeocystis")
print("="*80 + "\n")
print(df.to_string(index=False))
print("\n" + "="*80)
print(f"\nTotal proteins extracted: {len(results)}/4")
print(f"Results saved in: {RESULTS_DIR}")
print("="*80)


PROTEIN EXTRACTION SUMMARY - Psilocybe baeocystis

Gene                     Enzyme  Length (aa)  Reference (aa)  Difference  Stop codons Status
PsiD   Tryptophan decarboxylase          206             439        -233            0      ✅
PsiH Tryptamine 4-monooxygenase           61              69          -8            0      ✅


Total proteins extracted: 2/4
Results saved in: /Users/felix/Library/CloudStorage/OneDrive-SharedLibraries-MacquarieUniversity/Australian Genome Foundry - AWS cloud infrastructure/08_Ancestral_Seq_Sofia/20251015_Meeting_PsiloProteins_Felix/05_Felix_code/results


---

# Batch Process All Species

Now that we've validated the workflow on P. baeocystis, let's extract proteins from **all 71 species** in the dataset.

**Goal:** Extract all 4 BGC proteins from all species for phylogenetic analysis

---

## Setup Batch Processing

In [ ]:
# Setup paths for batch processing
SCAFFOLD_DIR = PROJECT_DIR / "03_Sequences_Paper" / "Assembly_scaffolds"

# Create batch results directory
BATCH_RESULTS_DIR = Path.cwd() / "batch_results"
BATCH_RESULTS_DIR.mkdir(exist_ok=True)

# Create subdirectories for each gene
for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    (BATCH_RESULTS_DIR / gene).mkdir(exist_ok=True)

# Get all scaffold files
scaffold_files = sorted(list(SCAFFOLD_DIR.glob("*.scaffolds.fasta")))

print(f"Scaffold directory: {SCAFFOLD_DIR}")
print(f"Batch results directory: {BATCH_RESULTS_DIR}")
print(f"\nFound {len(scaffold_files)} species scaffold files")
print(f"\nFirst 5 species:")
for i, scaffold_file in enumerate(scaffold_files[:5], 1):
    species_name = scaffold_file.stem.replace('.scaffolds', '')
    print(f"  {i}. {species_name}")

## Define Extraction Function

In [ ]:
def extract_protein(species_name, scaffold_file, gene, reference_protein, output_dir):
    """
    Extract a single protein from a genome scaffold.
    
    Returns:
        dict: Results with status, protein length, and any warnings
    """
    # Define output files
    output_cds = output_dir / f"{species_name}_{gene}.cds.fa"
    output_protein = output_dir / f"{species_name}_{gene}.prot.fa"
    
    results = {
        'species': species_name,
        'gene': gene,
        'status': 'pending',
        'protein_length': 0,
        'stop_codons': 0,
        'warnings': []
    }
    
    try:
        # Step 1: Extract CDS with exonerate
        cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{scaffold_file}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
        
        result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True, timeout=300)
        
        if result.returncode != 0:
            results['status'] = 'failed_exonerate'
            results['warnings'].append(f"Exonerate failed: {result.stderr[:100]}")
            return results
        
        # Check if CDS was found
        if not output_cds.exists() or output_cds.stat().st_size == 0:
            results['status'] = 'no_match'
            results['warnings'].append("No match found in genome")
            return results
        
        # Step 2: Translate to protein
        cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
        result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True, timeout=60)
        
        if result.returncode != 0:
            results['status'] = 'failed_translate'
            results['warnings'].append(f"Translation failed: {result.stderr[:100]}")
            return results
        
        # Step 3: Quality control
        if output_protein.exists():
            record = list(SeqIO.parse(output_protein, "fasta"))[0]
            sequence = str(record.seq)
            
            # Check for internal stop codons
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            results['protein_length'] = len(sequence)
            results['stop_codons'] = stop_count
            
            if stop_count > 0:
                results['warnings'].append(f"{stop_count} internal stop codon(s)")
            
            if len(sequence) < 50:
                results['warnings'].append(f"Protein very short: {len(sequence)} aa")
            
            results['status'] = 'success'
        else:
            results['status'] = 'failed_qc'
            results['warnings'].append("Protein file not created")
    
    except subprocess.TimeoutExpired:
        results['status'] = 'timeout'
        results['warnings'].append("Command timeout")
    except Exception as e:
        results['status'] = 'error'
        results['warnings'].append(f"Exception: {str(e)[:100]}")
    
    return results

print("✓ Extraction function defined")

## Batch Extract All Species (Parallelized)

**Note:** Using multiprocessing to process all 71 species × 4 genes = 284 extractions in parallel

**Estimated time:** ~10-15 minutes (using all CPU cores)

In [ ]:
# Wrapper function for multiprocessing
def process_species_gene_pair(args):
    """Process a single (species, gene) combination."""
    scaffold_file, gene, gene_info, output_dir = args
    species_name = scaffold_file.stem.replace('.scaffolds', '')
    return extract_protein(
        species_name=species_name,
        scaffold_file=scaffold_file,
        gene=gene,
        reference_protein=gene_info['reference'],
        output_dir=output_dir
    )

# Create all tasks (species × gene combinations)
tasks = []
for scaffold_file in scaffold_files:
    for gene, info in BGC_GENES.items():
        output_dir = BATCH_RESULTS_DIR / gene
        tasks.append((scaffold_file, gene, info, output_dir))

print(f"Processing {len(scaffold_files)} species × 4 genes = {len(tasks)} extractions")
print(f"Using {mp.cpu_count() - 1} CPU cores for parallel processing")
print(f"Estimated time: ~10-15 minutes (vs 30-60 min sequential)\n")

# Run in parallel with progress bar
n_workers = max(1, mp.cpu_count() - 1)  # Leave 1 core for system

with mp.Pool(processes=n_workers) as pool:
    all_results = list(tqdm(
        pool.imap(process_species_gene_pair, tasks),
        total=len(tasks),
        desc="Extracting proteins"
    ))

print("\n✓ Batch extraction complete!")

## Batch Summary Statistics

In [ ]:
# Create summary DataFrame
df_results = pd.DataFrame(all_results)

print("="*80)
print("BATCH EXTRACTION SUMMARY")
print("="*80)

# Overall stats
print(f"\nTotal extractions: {len(df_results)}")
print(f"Successful: {len(df_results[df_results['status'] == 'success'])}")
print(f"Failed: {len(df_results[df_results['status'] != 'success'])}")

# Stats by gene
print("\nSuccess rate by gene:")
for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    gene_data = df_results[df_results['gene'] == gene]
    success_count = len(gene_data[gene_data['status'] == 'success'])
    total = len(gene_data)
    pct = (success_count / total * 100) if total > 0 else 0
    print(f"  {gene}: {success_count}/{total} ({pct:.1f}%)")

# Protein length statistics
print("\nProtein length statistics (successful extractions):")
success_df = df_results[df_results['status'] == 'success']
for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    gene_data = success_df[success_df['gene'] == gene]['protein_length']
    if len(gene_data) > 0:
        print(f"  {gene}: mean={gene_data.mean():.0f} aa, min={gene_data.min()}, max={gene_data.max()}")

# Show failures
failed_df = df_results[df_results['status'] != 'success']
if len(failed_df) > 0:
    print(f"\nFailed extractions ({len(failed_df)}):")
    print(failed_df[['species', 'gene', 'status']].head(10).to_string(index=False))
    if len(failed_df) > 10:
        print(f"... and {len(failed_df) - 10} more")

print("\n" + "="*80)

# Save results to CSV
results_csv = BATCH_RESULTS_DIR / "extraction_summary.csv"
df_results.to_csv(results_csv, index=False)
print(f"\nResults saved to: {results_csv}")

## Combine Proteins by Gene

Create combined FASTA files for each gene (all species together) - ready for phylogenetic analysis

In [ ]:
print("Combining proteins by gene...\n")

for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    gene_dir = BATCH_RESULTS_DIR / gene
    combined_file = BATCH_RESULTS_DIR / f"{gene}_all_species.faa"
    
    # Collect all successful protein files
    protein_files = sorted(gene_dir.glob("*_prot.fa"))
    
    combined_records = []
    for prot_file in protein_files:
        try:
            record = list(SeqIO.parse(prot_file, "fasta"))[0]
            # Update ID to include species name
            species_name = prot_file.stem.replace(f"_{gene}.prot", "")
            record.id = f"{species_name}|{gene}"
            record.description = f"{gene} [{species_name}]"
            combined_records.append(record)
        except:
            pass  # Skip files that can't be read
    
    # Write combined file
    SeqIO.write(combined_records, combined_file, "fasta")
    print(f"✓ {gene}: {len(combined_records)} sequences → {combined_file.name}")

print("\n" + "="*80)
print("BATCH PROCESSING COMPLETE!")
print("="*80)
print(f"\nResults location: {BATCH_RESULTS_DIR}")
print("\nCombined FASTA files ready for downstream analysis:")
for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    combined_file = BATCH_RESULTS_DIR / f"{gene}_all_species.faa"
    if combined_file.exists():
        print(f"  - {combined_file.name}")
print("\nNext steps:")
print("  1. Multiple sequence alignment (MAFFT)")
print("  2. Phylogenetic tree building (IQ-TREE)")
print("  3. Ancestral sequence reconstruction (ASR)")